<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_herBERT_wiki_dataset_fraction_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HerBERT benchmark for wiki dataset 

In [1]:
# check available GPU
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 418.67, 15079 MiB


In [2]:
%%capture
!pip install datasets transformers

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.is_available()

True

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      labels.append(content[0])
      texts.append(" ".join(content[1:]))
  return texts, labels
  # # create a dataframe using texts and labels
  # trainDF = pd.DataFrame()
  # trainDF['text'] = texts
  # trainDF['label'] = labels
  # return trainDF

#raw    
corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset.txt'
#lemmas
# corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset_lemmas.txt'

texts, labels = load_corpora_to_dataframe(corpora)

# 1. load corpora
# 2. create training, test fractions
# 3. create specific No. per class fractions
# 4. create train, val sets
# 5. create new fresh model (tokenizer can stay the same)
# 6. tokenize and encode train, test 

In [6]:
data_df = pd.DataFrame()
data_df['labels'] = labels
data_df['texts'] = texts

train, test = train_test_split(data_df, test_size=.2)

In [7]:
# frac = train.groupby(['labels']).sample(n=200, replace=True)

In [8]:
import torch
from transformers import HerbertTokenizer, RobertaForSequenceClassification, EvalPrediction

tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=34)

Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream

In [9]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

def build_databases(train_data, test_data):
  # create train and validation dataset
  train_texts, val_texts, train_labels, val_labels = train_test_split(train_data['texts'].to_list(), train_data['labels'].to_list(), test_size=.2)

  max_length = 200
  train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
  val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)
  test_encodings = tokenizer(test_data['texts'].to_list(), truncation=True, padding=True, max_length=max_length)

  # encode labels
  train_labels = encoder.fit_transform(train_labels)
  val_labels = encoder.fit_transform(val_labels)
  test_labels = encoder.fit_transform(test_data['labels'].to_list())

  # build pyTorch dataset
  import torch

  class wikiDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
          item['labels'] = torch.tensor(self.labels[idx])
          return item

      def __len__(self):
          return len(self.labels)

  train_dataset = wikiDataset(train_encodings, train_labels)
  val_dataset = wikiDataset(val_encodings, val_labels)
  test_dataset = wikiDataset(test_encodings, test_labels)
  return train_dataset, val_dataset, test_dataset

# train_dataset, val_dataset, test_dataset = build_databases()


In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy = "epoch",
    num_train_epochs=4,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    # print(predictions[:10])
    # print(labels[:10])
    return metric.compute(predictions=predictions, references=labels)

def get_trainer(model, train_dataset, val_dataset):
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,             # evaluation dataset
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  return trainer


In [23]:
no_samples_per_class = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 400]
# no_samples_per_class = [10]

result = []

for n_sample in no_samples_per_class:
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print('training for n_sample=', n_sample)
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=34)
  dataset_fraction = train.groupby(['labels']).sample(n=n_sample, replace=True)
  train_dataset, val_dataset, test_dataset = build_databases(dataset_fraction, test)
  trainer = get_trainer(model=model, train_dataset=train_dataset, val_dataset=val_dataset)
  trainer.train()
  # trainer.evaluate()
  accuracy = trainer.predict(test_dataset)
  print(accuracy.metrics)
  result.append(accuracy.metrics['eval_accuracy'])

result


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 10
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,3.548100,3.564927,0.044118,0.777500,87.457000
2,3.525100,3.559212,0.029412,0.779100,87.281000
3,3.478300,3.549028,0.044118,0.781700,86.993000
4,3.391600,3.552078,0.044118,0.791200,85.943000


{'eval_loss': 3.5302677154541016, 'eval_accuracy': 0.037037037037037035, 'eval_runtime': 15.577, 'eval_samples_per_second': 88.399}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 20
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream

...

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.227900,1.037176,0.859069,10.591100,77.046000
2,0.426000,0.300084,0.927696,10.890000,74.931000
3,0.160800,0.238778,0.950980,10.977600,74.334000
4,0.106900,0.207596,0.958333,10.929300,74.662000


{'eval_loss': 0.34723424911499023, 'eval_accuracy': 0.9324618736383442, 'eval_runtime': 17.8898, 'eval_samples_per_second': 76.971}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 140
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.034300,0.656518,0.909664,12.512400,76.084000
2,0.291800,0.334210,0.932773,12.732100,74.772000
3,0.108000,0.241694,0.956933,12.759000,74.614000


RuntimeError: ignored